In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

In [2]:
# 카테고리 데이터
df = pd.read_excel('./data/kategory.xlsx')

In [3]:
df.columns = ['KAN_CODE', '전체분류', '현재분류명', '상품예시']

In [4]:
df

,KAN_CODE,전체분류,현재분류명,상품예시
0,1010101,가공식품>조미료>종합조미료>천연/발효조미료,천연/발효조미료,"다시다, 미원, 멸치가루, 버섯가루"
1,1010102,가공식품>조미료>종합조미료>식초,식초,"과일초, 발효초, 감식초, 사과식초, 현미식초, 홍초"
2,1010103,가공식품>조미료>종합조미료>천일염,천일염,"구운천일염, 꽃소금, 굵은소금, 가는소금, 볶은소금"
3,1010104,가공식품>조미료>종합조미료>가공염,가공염,"정제염, 맛소금, 허브솔트, 깨소금"
4,1010105,가공식품>조미료>종합조미료>설탕,설탕,"슈가파우더, 과당, 각설탕, 황설탕, 백설탕, 흑설탕"
...,...,...,...,...
3057,11150301,패션잡화>양말>유아동주니어양말/기타양말>유아동주니어양말,유아동주니어양말,"유아동양말, 주니어양말, 유아동캐릭터양말"
3058,11150399,패션잡화>양말>유아동주니어양말/기타양말>기타양말,기타양말,"방수양말, 리본양말, 배색양말"
3059,11160101,패션잡화>유니섹스신발>유니섹스신발류>실내화,실내화,"유니섹스실내화, 유니섹스패브릭실내화, 남녀실내화"
3060,11160199,패션잡화>유니섹스신발>유니섹스신발류>기타 유니섹스신발류,기타 유니섹스신발류,"유니섹스샌들, 유니섹스운동화, 유니섹스구두"


In [67]:
temp = pd.DataFrame(df['전체분류'].apply(lambda row: row.split('>')).tolist())
temp

,0,1,2,3
0,가공식품,조미료,종합조미료,천연/발효조미료
1,가공식품,조미료,종합조미료,식초
2,가공식품,조미료,종합조미료,천일염
3,가공식품,조미료,종합조미료,가공염
4,가공식품,조미료,종합조미료,설탕
...,...,...,...,...
3057,패션잡화,양말,유아동주니어양말/기타양말,유아동주니어양말
3058,패션잡화,양말,유아동주니어양말/기타양말,기타양말
3059,패션잡화,유니섹스신발,유니섹스신발류,실내화
3060,패션잡화,유니섹스신발,유니섹스신발류,기타 유니섹스신발류


In [59]:
for i, r in enumerate(temp[temp[3] == '티셔츠'].index, start=1):
    temp.iloc[r, 3] += str(i)

temp[temp[3].str.startswith('티셔츠')]

,0,1,2,3
690,일상용품,애완용품,애완 패션용품,티셔츠1
2583,의류,패션의류,여성의류,티셔츠2
2602,의류,패션의류,남성의류,티셔츠3
2614,의류,유아동의류,유아동의류,티셔츠4
2671,전문스포츠/레저,골프,골프의류,티셔츠5
2797,전문스포츠/레저,스키/보드,보드복,티셔츠6


In [6]:
all_cat = df['전체분류'].apply(lambda row: row.split('>')).tolist()
unique_cat = sorted(set().union(*all_cat))
cat2id = {c: i for i, c in enumerate(unique_cat)}
id2cat = {v: k for k, v in cat2id.items()}

In [56]:
with open('./data/prep/catid_map.json', 'w') as f:
    json.dump({'cat2id': cat2id, 'id2cat': id2cat}, f)

In [43]:
all_cat_id = [[cat2id[c] for c in cat] for cat in all_cat]

In [44]:
df_id = pd.DataFrame(all_cat_id)
droped = [df_id[[col, col + 1]].drop_duplicates().values for col in range(3)]
parent2child = pd.DataFrame(np.concatenate(droped), columns=['parent', 'child'])
parent2child

,parent,child
0,102,2635
1,102,2266
2,102,2822
3,102,1650
4,102,920
...,...,...
3733,2240,2239
3734,2240,548
3735,2210,1837
3736,2210,387


In [45]:
parent2child_label = parent2child.copy()
parent2child_label['parent'] = parent2child_label['parent'].apply(lambda id: id2cat[id])
parent2child_label['child'] = parent2child_label['child'].apply(lambda id: id2cat[id])
parent2child_label

,parent,child
0,가공식품,조미료
1,가공식품,유제품
2,가공식품,축산가공식품
3,가공식품,수산가공식품
4,가공식품,대용식재료
...,...,...
3733,유아동주니어양말/기타양말,유아동주니어양말
3734,유아동주니어양말/기타양말,기타양말
3735,유니섹스신발류,실내화
3736,유니섹스신발류,기타 유니섹스신발류


In [54]:
parent2child.to_csv('./data/prep/category.taxo', sep='\t', header=False, index=False)

# 진흥원과 상품 마스터 데이터 간에 겹치는게 있는지 확인

In [9]:
df_master = pd.read_csv('./data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|')

/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda110/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [74]:
item_master[['대분류명', '중분류명', '소분류명']].drop_duplicates()

,대분류명,중분류명,소분류명
0,자점,자점,자점
1,과자/빵,과자행사,비스켓행사(빼빼로)
20,생활잡화,문구/완구,필기구
24,간편식품,식재/반찬,냉장식재료
26,간편식품,간편기타,냉장안주류
...,...,...,...
47395,서비스,APP예약,APP예약(생활잡화)
47396,서비스,APP예약,APP예약(일상용품)
48057,비알콜음료,비알콜음료,비알콜맥주
48227,밥류,도시락,밀키트


In [61]:
all_cat_master = item_master[['대분류명', '중분류명', '소분류명']].values.tolist()
unique_cat_master = set().union(*all_cat_master)
unique_cat = set().union(*all_cat)

In [62]:
len(unique_cat), len(unique_cat_master)

(3427, 463)

In [63]:
len(unique_cat & unique_cat_master)

70

In [64]:
overlap = list((unique_cat & unique_cat_master))

In [65]:
for o in overlap:
    print(o, end=', ')

휴대폰배터리, 김밥, 각티슈, 스타킹, 화장품, 의약외품, 기타유가공품, 차류, 데오드란트, 자동차용품, 유제품, 기타면류, 조리빵, 위생용품, 두부, 스낵, 소주, 치약, 성인용기저귀, 일반서적, 패션잡화, 면도기, 우산, 파우치, 일상용품, 맥주, 주스, 조미료, 건강식품, 면도날, 주류, 시리얼, 가공식품, 아이스크림, 디지털/가전, 유가공품, 전자담배, 한과, 이어폰, 청소용품, 숙취해소음료, 탐폰, 담배, 위스키, 인스턴트커피, 양말, 접착용품, 와인, 저장장치, 신선식품, 기타냉동식품, 원두커피, 서적, 게임기/타이틀, 생수, 전통음료, 차음료, 생활용품, 요구르트, 음료, 기타가공식품, 식빵, 두유, 계란, 칫솔, 기타디지털/가전, 애완용품, 우유, 기타, 탄산수, 

In [60]:
item_master = pd.read_excel('./data/마스터+정보+및+데이터+정의서/02.상품마스터_20210305.xlsx')

In [76]:
item_master.keys()

Index(['PLU_CD', '상품명', '대분류명', '중분류명', '소분류명', '카테고리코드', '상품속성코드', '상품속성명',
       '판매가능여부', '시작일자', '종료일자', '사용여부', '상온저온유형코드', '상온저온유형명', '시즌상품유형코드',
       '시즌상품유형명', '행사상품유형코드', '행사상품유형명', '특별상품유형코드', '특별상품유형명', '낱개상품코드',
       '신상품주기코드', '신상품주기명', '신상품시작일자', '신상품종료일자', '점포발주여부', '점포발주가능일자',
       '발주권장유형코드', '발주권장유형명', '반품유형코드', '반품유형명', '점포발주입수수량', '공급가', '매가',
       '점포유통기한일수', 'TRANL_CD', 'LT', 'MON_YN', 'TUE_YN', 'WED_YN', 'THU_YN',
       'FRI_YN', 'SAT_YN', 'SUN_YN'],
      dtype='object')

In [75]:
item_master[item_master['대분류명'].isin(overlap) | item_master['중분류명'].isin(overlap) | item_master['소분류명'].isin(overlap)]

,PLU_CD,상품명,대분류명,중분류명,소분류명,카테고리코드,상품속성코드,상품속성명,판매가능여부,시작일자,...,점포유통기한일수,TRANL_CD,LT,MON_YN,TUE_YN,WED_YN,THU_YN,FRI_YN,SAT_YN,SUN_YN
28,8806075359157,자주)머그미니(빨),생활잡화,생활용품,주방/욕실용품,2142012001,401,일반,예,20140325,...,9999.0,011724,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
29,8806075309565,자주)보온병스텐200ml(파),생활잡화,생활용품,주방/욕실용품,2142012001,401,일반,예,20140325,...,9999.0,011724,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
30,8806075220600,자주)바디퍼프버블(파),생활잡화,생활용품,주방/욕실용품,2142012001,401,일반,예,20140327,...,9999.0,011724,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
31,3057640100675,롯데)볼빅500ml,음료,수입음료,수입생수,2082072001,401,일반,예,20140328,...,243.0,011035,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
34,8801047224730,동원)참기름김10매*12입(CVS),가공식품,대용식,김류,2102022004,401,일반,예,20140326,...,80.0,011695,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48954,8801045074184,오뚜기)도가니탕죽285g,가공식품,대용식,즉석죽,2102022001,401,일반,예,20210226,...,210.0,011758,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
48955,8801083453002,농심)켈로그크랜베리아몬드그래놀라300g,가공식품,대용식,시리얼,2102022008,401,일반,예,20210226,...,180.0,011689,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
48956,8801045226781,오뚜기)사골곰탕죽285g,가공식품,대용식,즉석죽,2102022001,401,일반,예,20210226,...,210.0,011758,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
48957,8801045903293,오뚜기)꽃게랑면큰컵(한정판),가공식품,면식품,용기면(대),2102012001,401,일반,예,20210226,...,75.0,010566,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [35]:
item_new = pd.read_excel('./data/마스터+정보+및+데이터+정의서/2020년부터 추가된 신상품 데이터 설명 포함.xlsx')

In [36]:
item_new[item_new['대분류'].isin(overlap) | item_new['중분류'].isin(overlap) | item_new['소분류'].isin(overlap)]

,공지년도,공지월,공지주차,공지일자,확정여부,PLU,상품명,대분류코드,대분류,중분류코드,...,취급구분,유통기한보관법,발주게시일,발주단위/입수,리드타임(일),발주주기,지금조건,금액(각),발주기간,정산월/지급횟수
18,2020,4,4,20200427,Y,8801068386394,샌드팜)new빅케이준치킨햄샌드위치2편,2020000000,조리빵,2022010000,...,NaN,2(일) / 저온,4월 28일,1개/0개(1BOX),1일,"주7회/월,화,수,목,금,토,일",개이상발주시,NaN,NaN,NaN
19,2020,4,4,20200427,Y,8801068386400,샌드팜)new빅케이준치킨햄샌드위치,2020000000,조리빵,2022010000,...,NaN,2(일) / 저온,4월 28일,1개/0개(1BOX),1일,"주7회/월,화,수,목,금,토,일",개이상발주시,NaN,NaN,NaN
20,2020,5,1,20200507,Y,8809631986231,현대)에그&멜팅치즈토스트샌드,2020000000,조리빵,2022010000,...,NaN,2(일) / 저온,5월 19일,1개/1개(1BOX),1일,"주7회/월,화,수,목,금,토,일",NaN,NaN,NaN,NaN
21,2020,5,1,20200507,Y,8809631986248,현대)에그&멜팅치즈토스트샌드2편,2020000000,조리빵,2022010000,...,NaN,2(일) / 저온,5월 19일,1개/1개(1BOX),1일,"주7회/월,화,수,목,금,토,일",NaN,NaN,NaN,NaN
22,2020,8,1,20200806,Y,8809650793537,대정)맛녀석에그듬뿍치아바타샌드,2020000000,조리빵,2022010000,...,NaN,2(일) / 저온,8월 11일,1개/1개(1BOX),1일,"주7회/월,화,수,목,금,토,일",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1877,2021,1,4,20210128,Y,8809681243056,해신)휘_핑크펀치,1130000000,담배,1131520000,...,NaN,24개월/상온,1월 28일,6개/144개(1BOX),1일,"주6회/월,화,수,목,금,토",6개이상발주시,"5,000원(얼리버드 8,000원)",01/28~02/27(얼리버드 01/28~02/03),02/1
1878,2021,2,4,20210225,Y,8809721710012,집랩스)ZSTICK전용종이집기,1130000000,담배,1131520000,...,NaN,1(일) / 상온,2월 25일,1개/1개(1BOX),1일,"주6회/월,화,수,목,금,토",NaN,NaN,NaN,NaN
1879,2021,1,1,20210107,Y,8809741170094,구어메이)버블몬몽스_리치메로,1130000000,담배,1131520000,...,NaN,유통기한 없음 / 상온,1월 7일,9개/225개(1BOX),1일,"주6회/월,화,수,목,금,토",9개이상발주시,"4,000원(얼리버드 6,000원)",01/07~01/20(얼리버드 01/07~01/13),01/1
1880,2021,1,1,20210107,Y,8809741170186,구어메이)버블몬몽스_멘솔,1130000000,담배,1131520000,...,NaN,유통기한 없음 / 상온,1월 7일,9개/225개(1BOX),1일,"주6회/월,화,수,목,금,토",9개이상발주시,"4,000원(얼리버드 6,000원)",01/07~01/20(얼리버드 01/07~01/13),01/1
